In [72]:
import os, random, csv, sys

limit = sys.maxsize
while True:
    try:
        csv.field_size_limit(limit)
        break
    except OverflowError:
        limit = int(limit/2)

In [94]:
class NaiveBayesClassifier():

    def __init__(self, alpha=1):
        ## Used for smoothing when we encounter a word we have not seen before
        self._alpha = alpha
        
        ## Establishing probability variables
        self._probSpam = -1
        self._probNotSpam = -1
        
        ## Establishing dictionaries to keep count of spam and ham word occurences
        self._spamWords = dict()
        self._hamWords = dict()
        
        ## Establishing variables to keep track of spam, ham, and total words
        self._totalSpamWords = 0
        self._totalHamWords = 0
        self._totalWords = 0
        

    ### Helper functions ###

    # Get the dictionary value for word
    def _getWordOccurences(self, word, d):
        if word in d.keys():
            return d[word] + self._alpha
        else:
            return self._alpha
    
    # Generator for getting all words in an email
    def _getEmailWords(self, email):
        for word in email[2].split():
            if word.isalpha():
                yield word
        return

    # Retrieves the text of an email
    def _getEmail(self, email):
        return email[2]

    # Determines whether or not a file is spam based on the file name
    def _emailIsSpam(self, email):
        return email[3] == '1'

    # Adds a word to the dictionary
    # If the word is not in the dictionary then initialize it with a value of one
    def _addWord(self, word, d):
        if word in d.keys():
            d[word] += 1
        else:
            d[word] = 1
        
        
    ## Trains classifier with the given dataSet 
    def train(self, dataSet):
        totalEmails = len(dataSet)
        spamEmails = 0
        for email in dataSet:
            isSpam = self._emailIsSpam(email)
            if isSpam:
                spamEmails += 1
            for word in self._getEmailWords(email):
                self._totalWords += 1
                if isSpam:
                    self._addWord(word, self._spamWords)
                    self._totalSpamWords += 1
                else:
                    self._addWord(word, self._hamWords)
                    self._totalHamWords += 1
        self._probSpam = spamEmails/totalEmails
        self._probNotSpam = (totalEmails - spamEmails)/totalEmails

                
    ## Gives the probability that an email body is spam given whether or not the email is assumed as spam
    ## This function can give us P(B | A) and P(B | Not A)
    def conditionalEmailProb(self, email, isSpam):
        probability = 1
        for word in self._getEmailWords(email):
            probability *= self.conditionalWordProb(word, isSpam)
        return probability


    ## Gives the probability that a word is spam given whether or not the email is assumed as spam
    ## P(B_word | A_email)
    def conditionalWordProb(self, word, isSpam):
        if isSpam:
            return self._getWordOccurences(word, self._spamWords)/self._totalSpamWords
        else:
            return self._getWordOccurences(word, self._hamWords)/self._totalHamWords
        
        
    ## Main function that classifies emails as spam or not spam
    def classify(self, email):
        isSpam = self.conditionalEmailProb(email, True)
        notSpam = self.conditionalEmailProb(email, False)
        return isSpam > notSpam
    
    def evaluate(self, testingData):
        correctSpam = 0
        correctHam = 0
        spam = 0
        ham = 0
        for email in testingData:
            Ypred = self.classify(email)
            Y = email[3] == '1'
            if Y:
                spam += 1
                if Ypred:
                    correctSpam += 1
            else:
                ham += 1
                if not Ypred:
                    correctHam += 1
        total = spam + ham
        print("Finished Testing on {} emails,".format(total), end=' ')
        print("{} spam and {} ham".format(spam, ham))
        print("\nSpam emails correctly identified: {}/{}".format(correctSpam,spam))
        print("Ham emails correctly identified: {}/{}".format(correctHam,ham))
        print("----------------------------------------")
        print("Correctly identified emails with {}% accuracy!".format((correctHam + correctSpam)/total*100))

In [95]:
data_set = []
with open("C:/Users/shous/Data/emailspam/train.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    i = 0
    for row in reader:
        data_set.append(row)

random.shuffle(data_set) #Randomize order of data set
dataSplit = round(len(data_set) * 0.7) #Used to split the data into 70% training and 30% testing
trainingData = data_set[:dataSplit]
testingData = data_set[dataSplit:]

In [96]:
classifier = NaiveBayesClassifier(0.1) # Establishes our classifier
trainingEmailNum = len(trainingData)
classifier.train(trainingData)
classifier.evaluate(testingData)

Finished Testing on 2505 emails, 660 spam and 1845 ham

Spam emails correctly identified: 272/660
Ham emails correctly identified: 1833/1845
----------------------------------------
Correctly identified emails with 84.03193612774452% accuracy!
